In [ ]:
import os
import sys
from scoring_functions import tanimoto
from utils import Variable, seq_to_smiles, unique
from rdkit import Chem
from rdkit.Chem import Draw
from scoring_functions import logP
import numpy as np
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from typing import List

In [ ]:

def generate(molecules, number_samples: int, seed) -> List[str]:
        np.random.seed(seed)
        return list(np.random.choice(molecules, size=number_samples))

In [ ]:
def score(smi):
    mol = Chem.MolFromSmiles(smi)
    if mol is None or len(smi) == 0:
        return 0
    else:
        smi = Chem.MolToSmiles(mol)

        score = 0
        alpha = 10.0 
        score_qed = float(self.evaluator(smi))
        t = tanimoto(self.query_structure)
        score_tanimoto = np.array(t.__call__([smi]))[0]
        beta = 10.0
        gamma = 1.0
        score += beta * score_tanimoto
        s= logP()
        score_logp = np.array(s.__call__([smi]))[0]
        weights = [alpha, beta, gamma]
        score = alpha*score_qed+beta*score_tanimoto+gamma*score_logp
        score = score / sum(weights)


            self.mol_buffer[smi] = [score, len(self.mol_buffer)+1]
    return self.mol_buffer[smi][0]



In [ ]:
dist_file = "data/ChEMBL_filtered.smi"

with open(dist_file, 'r') as smiles_file:
    smiles_list = [line.strip() for line in smiles_file.readlines()]

seeds = [0, 1, 2, 3, 5, 7, 11, 13, 17, 19]#, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]

for seed in seeds:
    
    random_100 = generate(smiles_list, 100, seed)
  